## Ejercicio 2

Utilice para entrenamiento y prueba los conjuntos de datos `concent_trn.csv` y `concent_tst.csv`, que consisten en dos clases distribuidas en forma concéntrica como muestra la Figura 1. Determine la estructura de una red de tipo perceptrón multicapa que resulte más apropiada para resolver este problema. Represente gráficamente, con diferentes colores, el resultado de la clasificación realizada por el perceptrón multicapa.

### Solución


In [ ]:
import numpy as np

arr = np.ndarray([1])
print(arr.shape)

(1,)
